# 단어가 ""로 바뀌거나 correct-error
### 돌리는 순서: update_wrong_error > ethics > coredata

In [46]:
import json
import re
from collections import Counter
import traceback

# 경로 설정
# 본인 outputJson 폴더 위치
outputJson= "C:\\workplace\\Kilab\\etri\\dataset\\취합\\평가셋\\003_KILab 재검수\\003_전체\\"
# outputJson = "C:\\workplace\\Kilab\\git\\caption_tool\\typescript\\front\\public\\json\\outputJson\\"
# outputJson = "C:\\workplace\\Kilab\\etri\\dataset\\취합\\학습셋\\002_에러캡션 완료\\all\\" # 학습셋
# output_path = f"C:\\workplace\\Kilab\\etri\\dataset\\취합\\평가셋\\003_KILab 재검수\\003_전체\\"



def filter_prepositions_and_articles(word_list):
    """
    @description
    리스트 내 단어에서 전치사 및 관사 제거
    find_diff_and_compound에서 사용용

    """
# 전치사와 관사 목록
    prepositions = {
        "in", 
        "on", 
        "at", 
        "by", 
        "with", 
        "with in", 
        "within", 
        "about",
        "against",
        "between",
        "into",
        "in to",
        "through",
        "during",
        "before",
        "after",
        "above",
        "below",
        "to",
        "from",
        "up",
        "down",
        "for",
        "over",
        "under",
        "again",
        "further",
        "off",
        "near",
    }
    articles = {"a", "an", "the"}

    # 각 구에서 전치사 및 관사 제거
    filtered_phrases = []
    for phrase in word_list:
        words = re.findall(r"\w+", phrase.lower())  # 각 구를 단어로 분리 (소문자 처리)
        words = phrase.split()  # 단어로 분리

        if words[0].lower() in prepositions:  # 첫 단어가 전치사면
            words = words[1:]  # 전치사 제거

        filtered_words = [word for word in words if word not in articles]
        filtered_phrases.append(" ".join(filtered_words))

    return filtered_phrases

def find_diff_and_compound(correct, error):
    """
    @description
    correct, error caption을 비교하여 달라진 부분 추출(복합명사 살려서)

    """
    # Step 1: 문장을 단어 리스트로 분리 (공백 및 구두점 처리)
    error_words = re.findall(r"\w+", error.lower())  # 문장1의 단어 목록 (소문자 처리)
    correct_words = re.findall(
        r"\w+", correct.lower()
    )  # 문장2의 단어 목록 (소문자 처리)

    # Step 2: 겹치는 단어 제거
    # 리스트의 요소 개수를 세기
    counter1 = Counter(error_words)
    counter2 = Counter(correct_words)

    common_words = counter1 & counter2  # 겹치는 단어 목록
    # 공통 단어를 제외한 error_words의 고유 단어들
    unique_error_words = list((counter1 - common_words).elements())

    # 공통 단어를 제외한 correct_words의 고유 단어들
    unique_correct_words = list((counter2 - common_words).elements())

    # Step 3: 겹치지 않는 단어들 중에서 문장에서 붙어 있는 단어 찾기
    def find_compound_nouns(words, original_sentence):
        compounds = []
        i = 0
        while i < len(words):
            # 복합 명사 조합을 체크
            found_compound = False
            for j in range(len(words), 0, -1):  # len(words)개까지 조합 가능하도록
                if i + j <= len(words):
                    compound_candidate = " ".join(words[i : i + j])
                    if compound_candidate in original_sentence:
                        compounds.append(compound_candidate)
                        i += j  # 조합한 단어 개수만큼 인덱스 증가
                        found_compound = True
                        break
            if not found_compound:  # 복합 명사가 아닐 경우
                compounds.append(words[i])
                i += 1
        return compounds

    # 문장1과 문장2에서 각각 복합 명사 추출
    human_annotation = find_compound_nouns(unique_error_words, error.lower())
    correct_keyword = find_compound_nouns(unique_correct_words, correct.lower())
    filtered_human_annotation = filter_prepositions_and_articles(human_annotation)
    filtered_correct_keyword = filter_prepositions_and_articles(correct_keyword)


    # 최종 차이점 리스트
    return filtered_human_annotation, filtered_correct_keyword

import os


def load_wrong_data(file_num):
    """
    @description
    주어진 경로에서 JSON 파일을 불러와 비윤리적 단어로 변경되었을 가능성이 있는 데이터 확인
    """
    try :
        file_path = outputJson+f'output_{file_num}.json'
        print(os.path.exists(file_path))
        
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # 박스를 캡션 수를 기준으로 내림차순
        sorted_boxes = sorted(data['new_bounding_boxes'],key=lambda x: len(x['captions']), reverse=True)
       
        for box in sorted_boxes:
            # 캡션 세트 리스트 가져오기
            for captions in box['captions']:
                if captions['errorCaption'] == [] or "time reads 6:05 pm" in captions['errorCaption'][0]:
                    return file_num
                elif captions['caption'] == captions['errorCaption'][0]:
                    print(f"correct-error 값이 같음: {file_num}")
                    return file_num
                elif bool(re.search(r'[\uAC00-\uD7A3]', captions['errorCaption'][0])):
                    print(file_num,":", captions['errorCaption'][0])
                    return file_num
                
                errors, corrects = find_diff_and_compound(captions['caption'], captions['errorCaption'][0])  
                if errors == []:
                    print(captions['caption'], captions['errorCaption'][0], file_num)
                    return file_num
    except :
        print("json 데이터 읽는 중 오류 발생::", traceback.format_exc())
        return None
        

In [47]:
# wrong_data = [load_wrong_data(num) for num in range(779,1175) if load_wrong_data(num)!=None] # <--완료 송강현 549~778
wrong_data = [load_wrong_data(num) for num in range(1559,1823) if load_wrong_data(num)!=None] # 송강현 549~778
# wrong_data = [i for i in wrong_data]
print(wrong_data)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
1702 : black tinted window on 버스
True
1702 : black tinted window on 버스
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

In [48]:
from error_dict import dictionary 

# 대체어 그룹 정의
def merge_dicts(*dicts):
    merged_dict = {}
    for dictionary in dicts:
        for key, value in dictionary.items():
            if key in merged_dict:
                print(key, value)
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = value
    return merged_dict

replacement_groups = merge_dicts(dictionary)

# 정규 표현식 패턴 생성
replacement_patterns = {
    replacement: re.compile(r"\b" + r"\b|\b".join(re.escape(term) for term in terms) + r"\b", re.IGNORECASE)
    for replacement, terms in replacement_groups.items()
}

# JSON 파일 읽기
for i in wrong_data:# wrong_data: # (각자 구축량 시작점, 각자 구축량 마감개수 + 1) 
    file_path = outputJson+f'output_{i}.json'
    print(i)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    new_bounding_boxes = data.get('new_bounding_boxes', [])

    for box_index, box in enumerate(new_bounding_boxes):
        for caption_idx, caption in enumerate(box['captions']):
            if caption['errorCaption'] == []:
                new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'] = [caption['caption'].lower().strip()]
                for replacement, pattern in replacement_patterns.items():
                    if pattern.search(caption['caption'].lower().strip()):
                        new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'] = [pattern.sub(replacement, new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][0])]
                        
                if new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][0] == caption['caption'].lower().strip():
                    print(f'에러없음 - 안바뀜:', new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'])
                else:
                    print(f'채운 에러:', new_bounding_boxes[box_index]['captions'][caption_idx]['caption'], "\n", new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'])
            else:
                for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                    correct_caption = caption['caption'].lower().strip()
                    error_caption = errorCaption.lower().strip()
                    '''완전 일치'''
                    if caption['caption'].lower().strip() == errorCaption.lower().strip():
                        for replacement, pattern in replacement_patterns.items():
                            if pattern.search(error_caption):
                                # 대체어로 변경
                                new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = pattern.sub(replacement, error_caption)
                                break
                        if new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] == error_caption:
                            print(f'완전일치 - 안바뀜:',new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx])
                        else:
                            print(f'바뀐 에러:', new_bounding_boxes[box_index]['captions'][caption_idx]['caption'], "\n", new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][0])
                            
                    else:
                        errors, corrects = find_diff_and_compound(correct_caption, error_caption) 
                        # print(errors)
                        '''공백이 들어간 경우'''
                        if errors == []:
                            for replacement, pattern in replacement_patterns.items():
                                if pattern.search(caption['caption'].lower().strip()) is None:
                                    new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = pattern.sub(replacement, correct_caption)
                                    break
                                    # print(f'바뀜: {new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx]}')
                            if new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] == [correct_caption]:
                                print(f'공백이 들어감 - 안바뀜: {errorCaption}')
                        # else:
                        #     if "time reads 6:05 pm" in error_caption:
                        #         new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][0] = caption['caption']

    data['new_bounding_boxes'] = new_bounding_boxes
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

1702
1796
1806
바뀐 에러: the ball is yellow 
 the dice is yellow
바뀐 에러: it is a green ball 
 it is a green dice
완전일치 - 안바뀜: the w is pink
완전일치 - 안바뀜: nose
바뀐 에러: left hand displayed upright. 
 left Handkerchief displayed upright.
바뀐 에러:  Front side of image there is a pole 
 front side of image there is a pipe
1822
바뀐 에러:  Background of image there are few plants 
  Background of image there are few plants
바뀐 에러:  Bottom of image is a grassy land 
  Bottom of image is a grassy land


In [1]:
# core 데이터 업데이트 확인
core_path="C:\\workplace\\Kilab\\etri\\dataset\\취합\\학습셋\\003_core 데이터 배포 버전\\all\\"

import json
for file_num in range(8744,11365):
    
    
    try:
        with open(core_path+f"core_{file_num}.json",'r', encoding='utf-8') as file:
            data = json.load(file)
        key = list(data.keys())[0]
        regions = data[key]['regions']
        unprocessed_keywords = data[key]['unprocessed_keywords']
        for region in regions:
            captions = region['captions']
            for caption_pairs in captions:
                if "time reads 6:05 pm" in caption_pairs['counterfactual_caption'] and "time reads 6:05 pm" != caption_pairs['counterfactual_caption']:
                    print("file = ", file_num)
                    print("ㅈ됨: ",caption_pairs['counterfactual_caption'])
    except:
        pass